In [0]:
import os
#os.chdir('Devnagri Detection')

In [0]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [0]:
train_dir = 'drive/Devnagri Detection/numerals'
test_dir = 'Dataset/Devnagri_number'

In [26]:
folders= ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for folder in folders:
    print('{} {} images'.format(folder, len(os.listdir(os.path.join(train_dir, folder)))))

0 298 images
1 288 images
2 288 images
3 288 images
4 288 images
5 288 images
6 288 images
7 288 images
8 288 images
9 288 images


In [27]:
train = []

for folder_num, folder in enumerate(folders):
    for file in os.listdir(os.path.join(train_dir, folder)):
        train.append(['drive/Devnagri Detection/numerals/{}/{}'.format(folder, file), folder_num, folder])
        
train = pd.DataFrame(train, columns=['file', 'folder_num', 'folder'])

print('Training Data: ',train.shape)

Training Data:  (2890, 3)


In [10]:
train.head()

,file,folder_num,folder
0,drive/Devnagri Detection/numerals/0/008_02.jpg,0,0
1,drive/Devnagri Detection/numerals/0/008_03.jpg,0,0
2,drive/Devnagri Detection/numerals/0/010_01.jpg,0,0
3,drive/Devnagri Detection/numerals/0/008_08.jpg,0,0
4,drive/Devnagri Detection/numerals/0/008_07.jpg,0,0


In [11]:
c= train['file'][1]
print(c)

img = cv2.imread('drive/Devnagri Detection/numerals/0/008_03.jpg')


drive/Devnagri Detection/numerals/0/008_03.jpg


In [12]:
print(img)

[[[250 250 250]
  [255 255 255]
  [255 255 255]
  ...
  [252 252 252]
  [252 252 252]
  [252 252 252]]

 [[255 255 255]
  [255 255 255]
  [252 252 252]
  ...
  [249 249 249]
  [249 249 249]
  [249 249 249]]

 [[255 255 255]
  [252 252 252]
  [250 250 250]
  ...
  [252 252 252]
  [252 252 252]
  [252 252 252]]

 ...

 [[255 255 255]
  [255 255 255]
  [224 224 224]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[250 250 250]
  [246 246 246]
  [253 253 253]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [252 252 252]
  [248 248 248]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [13]:
x_train = []

for i in range(len(train)):
    img = cv2.imread(train['file'][i],0)
    img = cv2.resize(img,dsize=(36,36))
    bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    lower = np.array([0,0,0])
    upper = np.array([255,255,255])
    mask = cv2.inRange(hsv, lower, upper)
    res = cv2.bitwise_and(img,img, mask= mask)
    #kernel = np.ones((5,5),np.uint8)
    #kernel2 = np.ones((3,3),np.uint8)
    
    ret,thresh1 = cv2.threshold(res,150,255,cv2.THRESH_BINARY)
    laplacian = cv2.Laplacian(thresh1,cv2.CV_64F)
    #sobelx = cv2.Sobel(thresh1,cv2.CV_64F,1,0,ksize=5)
    #sobely = cv2.Sobel(thresh1,cv2.CV_64F,0,1,ksize=5)

    #ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create()
    #ridges = ridge_filter.getRidgeFilteredImage(thresh1)
    
    thresh1 = np.reshape(thresh1,(36,36,1))
    laplacian = np.reshape(laplacian,(36,36,1))
    #ridges = np.reshape(ridges,(224,224,1))
    
    x_train.append(np.concatenate((np.array(thresh1),np.array(laplacian)),axis=2))
    
    if ((i % 1000) == 0):
        print(i)

0
1000
2000


In [28]:
x_train = []

for i in range(len(train)):
    img = cv2.imread(train['file'][i],0)
    img = np.reshape(img,(36,36,1))
    x_train.append(img)
    if ((i % 500) == 0):
        print(i)

0
500
1000
1500
2000
2500


In [29]:
x_train = np.array(x_train)
x_train[1].shape

(36, 36, 1)

In [30]:
y_train = train['folder_num']
y_train = to_categorical(y_train, num_classes = len(folders))
print(y_train[1])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=10)

In [32]:
input_shape = x_train[1].shape
print('Input Shape is :', input_shape)

Input Shape is : (36, 36, 1)


In [45]:
"""def ConMax1(inputs, filters):
    x= Conv2D(filters, (3,3), activation='relu', padding='same')(inputs)
    x= Conv2D(filters, (3,3), activation='relu', padding='same')(x)
    out= MaxPooling2D((2, 2), strides=(1, 1), padding='same')(x)
    return out

def ConMax2(inputs, filters):
    x= Conv2D(filters, (3,3), activation='relu', padding='same')(inputs)
    x= Conv2D(filters, (3,3), activation='relu', padding='same')(x)
    x= Conv2D(filters, (3,3), activation='relu', padding='same')(x)
    out= MaxPooling2D((2, 2), strides=(1, 1), padding='same')(x)
    return out
    
def full_layer(inpust, neurons, dropout):
    x = Dense(neurons, activation='relu')(inpust)
    out = Dropout(dropout)(x)
    
image_input=Input(shape=input_shape)
ip = ConMax1(image_input, 64)
ip = ConMax1(ip, 128)
ip = ConMax2(ip, 256)
ip = ConMax2(ip, 512)
ip = ConMax2(ip, 512)
ip = Flatten()(ip)

ip = Dense(256, activation='relu')(ip)
ip = Dropout(0.5)(ip)
ip = Dense(256, activation='relu')(ip)
ip = Dropout(0.5)(ip)
out = Dense(len(folders), activation='softmax')(ip)

model_new3 = Model(image_input, out)
model_new3.summary()
"""
image_input=Input(shape=input_shape)
x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(image_input)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(x)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(image_input)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(x)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(image_input)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(x)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(image_input)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x = Conv2D(32, (5, 5), activation= 'tanh', padding ='valid')(x)
x= MaxPooling2D((2,2), strides=(1, 1), padding='same')(x)

x= Flatten()(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)

out = Dense(len(folders), activation='softmax')(x)

model_new3 = Model(image_input, out)
model_new3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 36, 36, 1)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 32)        832       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 28, 28, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               6422784   
__________

In [0]:
model_new3.compile(optimizer = Adam(lr=.001) , loss = 'categorical_crossentropy', metrics=['accuracy'])

In [47]:
model_new3.fit(x_train, y_train, epochs=40, batch_size=50)

Epoch 1/40
2312/2312 [==============================] - 2s 1ms/step - loss: 2.0966 - acc: 0.5640
Epoch 2/40
2312/2312 [==============================] - 1s 483us/step - loss: 0.4948 - acc: 0.8408
Epoch 3/40
2312/2312 [==============================] - 1s 472us/step - loss: 0.3606 - acc: 0.8824
Epoch 4/40
2312/2312 [==============================] - 1s 511us/step - loss: 0.3683 - acc: 0.8919
Epoch 5/40
2312/2312 [==============================] - 1s 471us/step - loss: 0.2785 - acc: 0.9178
Epoch 6/40
2312/2312 [==============================] - 1s 473us/step - loss: 0.2264 - acc: 0.9360
Epoch 7/40
2312/2312 [==============================] - 1s 472us/step - loss: 0.1666 - acc: 0.9533
Epoch 8/40
2312/2312 [==============================] - 1s 477us/step - loss: 0.1481 - acc: 0.9606
Epoch 9/40
2312/2312 [==============================] - 1s 471us/step - loss: 0.2107 - acc: 0.9494
Epoch 10/40
2312/2312 [==============================] - 1s 470us/step - loss: 0.2467 - acc: 0.9407
Epoch 11/40

In [48]:
y_val_pred = model_new3.evaluate(x_val, y_val, batch_size=32, verbose=1, sample_weight=None)

578/578 [==============================] - 0s 495us/step


In [49]:
print ("Validation Loss = " + str(y_val_pred[0]))
print ("Validation Accuracy = " + str(y_val_pred[1]))

Validation Loss = 0.2815931026897736
Validation Accuracy = 0.9653979238754326
